In [6]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# let us import some useful things
from lib import *
from classifiers import *
import math
from scipy.stats import ttest_ind
from scipy.special import stdtr
import numpy as np

In [8]:
# load the data same as before
microarray_file_name = '../data/leukemia_ALL_AML_matrix.txt'
labels_file_name = '../data/leukemia_ALL_AML_labels.txt'
data_store = DataSet(microarray_file_name, labels_file_name)

In [9]:
train_data_set = data_store.get_train_set() 

# separate samples based on label
ALL_gene_profiles_list = [s.get_gene_profile() for s in train_data_set if s.get_label() == 1]
AML_gene_profiles_list = [s.get_gene_profile() for s in train_data_set if s.get_label() == 0]


# let's select our most useful genes based on the Welch's t-test!
init_number_of_genes = len(train_data_set[0].get_gene_profile())
selected_genes = []
for gene_id in range(init_number_of_genes):
    gene_values_ALL_samples = [profile[gene_id] for profile in ALL_gene_profiles_list]
    gene_values_AML_samples = [profile[gene_id] for profile in AML_gene_profiles_list]
    
    # TODO: use the formulas we've seen to compute the t-test value and the degree of freedom for the ALL and AML vector
    # hint: to compute the mean and variance of each list -- use the np functions mean() and var()!
    
    t_test_val = 0 # TODO: change this line!
    degree_of_freedom = 0 # TODO: change this line!

    # now we can use an existing funtion to find the p-value (don't worry about this!)
    # it is same as looking-up the t-test table!
    #p_value = 2*stdtr(degree_of_freedom, -np.abs(t_test_val))
    p_value = ttest_ind(gene_values_ALL_samples, gene_values_AML_samples)[1]
    
    if(p_value < 0.05):
        selected_genes.append(gene_id)

print ('Number of informative genes: ' + str(len(selected_genes)))

# now let's update our dataset to ignore the genes that are not informative
filtered_train_samples = []
filtered_test_samples = []
full_data_set = data_store.get_train_set() + data_store.get_test_set()
for sample in data_store.get_train_set():
    new_sample = Sample(sample.get_label(), [sample.get_gene_profile()[i] for i in range(init_number_of_genes) if i in selected_genes])
    filtered_train_samples.append(new_sample)
for sample in data_store.get_test_set():
    new_sample = Sample(sample.get_label(), [sample.get_gene_profile()[i] for i in range(init_number_of_genes) if i in selected_genes])
    filtered_test_samples.append(new_sample)  



Number of informative genes: 1234


Do you think we can change the p_value cut_off a bit?

In [12]:
# we will do a simple Logistic regression here
from sklearn.linear_model import LogisticRegression
train_set = data_store.get_train_set()
test_set = data_store.get_test_set()
train_profile = [x.get_gene_profile() for x in train_set]
train_labels = [x.get_label() for x in train_set]

clf = LogisticRegression().fit(train_profile, train_labels)
#now we predict
test_profile = [x.get_gene_profile() for x in test_set]
test_labels = [x.get_gene_profile() for x in test_set]
pred = clf.predict(test_profile)
print(pred)
evaluate_results(list(zip(test_set, pred)))

#clf_filtered = LogisticRegression().fit(filtered_test_samples, y)

[0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0]
Accuracy: 96.00%


In [ ]:
#TODO: try knn using the filtered genes again, any change in accuracy?